<a href="https://colab.research.google.com/github/aicharya/HomePrice/blob/master/KaggleHomePrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Checkout the repo

Pull the latest files

Forward Stepwise Regression

Ridge Regression

Lasso Regression

Regression Tree

The Optimal Model